<img src='https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcQ-VfNtOyJbsaxu43Kztf_cv1mgBG6ZIQZEVw&usqp=CAU'>

# Procesamiento de Lenguaje Natural

## Taller #12: Análisis de sentimiento
`Fecha de entrega: Mayo 27, 2021 (Antes del inicio de la próxima clase).`

`Modo de entrega: Subir link de GitHub al aula virtual.`

In [2]:
import pandas as pd 
import tweepy

In [3]:
keys = pd.read_csv("keys.csv", header=None)

In [4]:
# Leer las llaves
keys = dict(zip(keys[0],keys[1]))
keys.keys()

dict_keys(['api_key', 'api_secret_key', 'access_token', 'access_token_secret', 'bearer_token'])

In [5]:
# Autenticar a través de la API 
auth = tweepy.OAuthHandler(keys['api_key'], keys['api_secret_key'])
auth.set_access_token(keys['access_token'], keys['access_token_secret'])
api = tweepy.API(auth, wait_on_rate_limit=True)

In [6]:
#TEMA: COVID 19

tweets = tweepy.Cursor(api.search,
                       q=['covid 19 -RT'],
                       lang='es',
                       geocode=f'4.7110,-74.0721,100km',
                       tweet_mode='extended',
                       include_rts=False).items(100)

In [7]:
# Objeto generador
t = [tweet for tweet in tweets]
#t[0]

In [8]:
t_processed = []

for tweet in t:
    temp = []
    temp.append(tweet.full_text)
    temp.append(tweet.created_at)
    temp.append("Bogotá")
    temp.append(tweet.user.location)
    temp.append(tweet.id)
    temp.append(f"https://twitter.com/i/web/status/{tweet.id}")
    
    t_processed.append(temp)

In [9]:
data = pd.DataFrame(t_processed)
data.columns = ['tweet', 'date', 'city', 'user_location', 'id', 'link']
data

,tweet,date,city,user_location,id,link
0,"En Colombia, 21 millones de personas están en ...",2021-05-21 00:13:17,Bogotá,"Bogotá, Colombia",1395533116291391495,https://twitter.com/i/web/status/1395533116291...
1,@pla219 @stevenarce Santa Fe anda igual o peor...,2021-05-21 00:12:37,Bogotá,"Bogotá, D.C., Colombia",1395532950842785796,https://twitter.com/i/web/status/1395532950842...
2,Boyacá contará con 12 nuevas UCI\n\nLas camas ...,2021-05-21 00:11:25,Bogotá,Tunja - Boyacá,1395532646797697026,https://twitter.com/i/web/status/1395532646797...
3,🔴 URGENTE - Argentina anuncia un cierre estric...,2021-05-21 00:05:35,Bogotá,"Bogotá, DC, Colombia",1395531180108222464,https://twitter.com/i/web/status/1395531180108...
4,¡A estos puntos solo deben asistir los mayores...,2021-05-21 00:05:08,Bogotá,"Bogotá, Colombia",1395531064815276034,https://twitter.com/i/web/status/1395531064815...
...,...,...,...,...,...,...
95,"A través de la resolución 524 de 2021, la ADRE...",2021-05-20 20:49:00,Bogotá,"Bogotá, D.C., Colombia",1395481706312912896,https://twitter.com/i/web/status/1395481706312...
96,.. Eso y las inexistentes reformas que logran ...,2021-05-20 20:47:44,Bogotá,Bogotá,1395481389475278852,https://twitter.com/i/web/status/1395481389475...
97,La OMS recordó que todas las vacunas disponibl...,2021-05-20 20:45:03,Bogotá,"Bogotá, Colombia",1395480711969259524,https://twitter.com/i/web/status/1395480711969...
98,@AlcaldiaBGA @JCardenasRey @vanguardiacom @LAF...,2021-05-20 20:35:08,Bogotá,"Bogotá, D.C., Colombia",1395478219474751490,https://twitter.com/i/web/status/1395478219474...


###  `[25 pts]` Punto 1: Hacer análisis de sentimiento de los datos del anterior taller
- Columna con el puntaje numérico del sentimiento
- Pueden usar cualquiera de las librerias vistas en clase

In [10]:
from textblob import TextBlob
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
vader = SentimentIntensityAnalyzer()

In [11]:
pd.set_option('display.max_colwidth', None)

data['sent_vader'] = data['tweet'].apply(lambda val: vader.polarity_scores(val)['compound'])
data['sent_textblob'] = data['tweet'].apply(lambda val: TextBlob(val).sentiment.polarity)
data[['tweet', 'sent_vader', 'sent_textblob']].head()

data[['tweet', 'sent_vader']].sort_values('sent_vader').tail()

,tweet,sent_vader
72,El uso del olfato de los perros para la detección de enfermos de coronavirus se ha extendido en los últimos meses en muchos países. https://t.co/vG27ttFNW2 vía @ELTIEMPO,0.3400
16,"2. Según Yamit Noe Hurtado gerente del centro asistencial, durante la pandemia de la Covid-19 este Hospital ha aumentado en un 600% su capacidad en unidades de cuidados intensivos pues pasó de tener 18 camas UCI en marzo del 2020 a contar con 108 en mayo del 2021.",0.3400
26,"@ClaudiaLopez Quién sabe qué gas será, y cuál exilio. Lo que sí es cierto es q o ha tenido ""empatía"" (???) con las pequeñas empresas ni con los católicos..Los ha perseguido pretextando el covid-19",0.6531
75,"https://t.co/9VWxlPjAcG Para el miércoles 19 de mayo las asignaciones de Gas Natural fueron de 896 GBTUD. Para el sector No Regulado, sin incluir los consumos de gas para la generación térmica, las asignaciones de Gas Natural fueron de 529 GBTUD. https://t.co/5LrV4eYFjV",0.6931
34,@powerhdeleon Lo bueno es que todos están vacunados contra el COVID-19 😂😂😂,0.8271


###  `[25 pts]` Punto 2: ¿Cuál es el tuit más positivo y más negativo?

In [19]:
# Más positivo
maxs = data.max()
maxs

tweet            😷COVID-19 en Bogotá hoy🦠 \n\n👉🏻Mortalidad diaria aumentó: pasamos de 103 fallecidos ayer a 113 fallecidos hoy, de los 490 registrados en el país.\n\n👉🏻Bogotá suma 4.917 nuevos contagios. \n\n👉🏻Ocupación Camas UCI para Covid-19 en Bogotá bajó a 95,3%. https://t.co/GynFs75DMW
date                                                                                                                                                                                                                                                                            2021-05-21 00:13:17
city                                                                                                                                                                                                                                                                                         Bogotá
user_location                                                                                                               

In [20]:
#Más negativo
min = data.min()
min

tweet            #ATENCION : Colombia no realizará la #CopaAmerica2021 El Covid-19 y la situación que vive el país le quita la posibilidad de realizar el certamen deportivo más importante del continente. #ParoNacional  https://t.co/dOgT9hB2qN
date                                                                                                                                                                                                                           2021-05-20 20:34:37
city                                                                                                                                                                                                                                        Bogotá
user_location                                                                                                                                                                                                                                     
id                          